<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Sentiment Analysis Using Vantage and LLM
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial'><b>Introduction:</b></p>

<p style = 'font-size:16px;font-family:Arial'>Sentiment analysis using <b>Teradata Vantage</b> and the advanced <b>AWS Bedrock - Anthropic's Claude LLM model</b> model involves leveraging cutting-edge technologies to extract insights from unstructured data. This process empowers businesses to swiftly identify and address customer concerns, enhancing overall customer satisfaction and loyalty.</p>

<p style = 'font-size:16px;font-family:Arial'><b>Key Features:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li><b>Polarity Classification:</b> Identifies specific emotions such as happiness, anger, sadness, and more.</li>
    <li><b>Emotion Detection:</b> The system comprehends the nuances of customer feedback, capturing subtle differences in tone and language.</li>
    <li><b>Aspect-Based Sentiment Analysis:</b>  Analyzes sentiment towards specific features or aspects of a product or service.</li>
    <li><b>Fine-Grained Sentiment Analysis:</b> Provides detailed sentiment analysis at the phrase or clause level.</li>
    <li><b>Subjectivity Classification:</b> Distinguishes between objective and subjective text.</li>

</ul>


<p style = 'font-size:16px;font-family:Arial'><b>Benefits:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li><b>Improved Customer Satisfaction:</b> Enhances customer experience by addressing concerns and improving products.</li>
    <li><b>Competitive Advantage:</b> Provides valuable insights to stay ahead of competitors.</li> 
    <li><b>Objective Insights:</b> Offers unbiased and accurate sentiment analysis.</li>
    <li><b>Real-Time Decision Making:</b> Enables swift responses to customer concerns and market trends.</li>
    <li><b>Scalability:</b> Handles large volumes of data efficiently.</li>
</ul>

<p style = 'font-size:16px;font-family:Arial'>Experience the transformative power of Generative AI in complaints classification.</p>

<p style = 'font-size:16px;font-family:Arial'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial'>
    <li>Configuring the environment</li>
    <li>Connect to Vantage</li>
    <li>Configuring AWS Bedrock - Anthropic's Claude LLM model</li>
    <li>Complaints Sentiment Analysis</li>
    <li>Cleanup</li>
</ol>

<hr style='height:2px;border:none;'>
<b style = 'font-size:20px;font-family:Arial'>1. Configuring the environment</b>
<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.1 Downloading and installing additional software needed</b>

In [ ]:
%%capture
!pip install -r requirements.txt --upgrade --quiet

In [ ]:
%%capture
#upgrade numpy to support wordcloud
!pip install --upgrade numpy matplotlib --quiet

<div class="alert alert-block alert-info">
    <p style = 'font-size:16px;font-family:Arial'><b>Note: </b><i>Please restart the kernel after executing these two lines. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>

<hr style="height:1px;border:none;">
<p style = 'font-size:18px;font-family:Arial'><b>1.2 Import the required libraries</b></p>
<p style = 'font-size:16px;font-family:Arial'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
# Data manipulation and analysis
import numpy as np
import pandas as pd
import getpass

# Set display options for dataframes, plots, and warnings
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
display.suppress_vantage_runtime_warnings = True

# Visualization
import plotly.express as px
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Progress bar
from tqdm import tqdm

# Machine learning and other utilities from Teradata
from teradataml import *
from sqlalchemy import func
from teradatagenai import TeradataAI, TextAnalyticsAI, VSManager, VectorStore, VSApi

# Requests
import requests

# Display settings
display.max_rows = 5
pd.set_option('display.max_colwidth', None)

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>2. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell.</p>

In [ ]:
print("Checking if this environment is ready to connect to VantageCloud Lake...")

if os.path.exists("/home/jovyan/JupyterLabRoot/VantageCloud_Lake/.config/.env"):
    print("Your environment parameter file exist.  Please proceed with this use case.")
    # Load all the variables from the .env file into a dictionary
    env_vars = dotenv_values("/home/jovyan/JupyterLabRoot/VantageCloud_Lake/.config/.env")
    # Create the Context
    eng = create_context(host=env_vars.get("host"), username=env_vars.get("username"), password=env_vars.get("my_variable"))
    execute_sql('''SET query_band='DEMO=text_analytics_teradatagenai_aws_huggingface.ipynb;' UPDATE FOR SESSION;''')
    print("Connected to VantageCloud Lake with:", eng)
else:
    print("Your environment has not been prepared for connecting to VantageCloud Lake.")
    print("Please contact the support team.")

<p style = 'font-size:16px;font-family:Arial'>Begin running steps with Shift + Enter keys. </p>

<hr style='height:2px;border:none'>
<p style = 'font-size:20px;font-family:Arial'><b>2.  Set up the LLM connection</b></p>

<p style = 'font-size:16px;font-family:Arial'>The <b>teradatagenai</b> python library can both connect to cloud-based LLM services as well as instantiate private models running <b>at scale</b> on local GPU compute. In this case we will use anthropoc claude-instant-v1 for low-cost, high-throughput tasks.</p>

<ol style = 'font-size:16px;font-family:Arial'>
<li><b>aws_access_key_id</b>: Enter your AWS access key ID</li>
<li><b>aws_secret_access_key</b>: Enter your AWS secret access key</li>
<li><b>region name</b>: Enter the AWS region you want to configure (e.g., us-east-1)</li>
<ol>

In [ ]:
access_key = getpass.getpass('aws_access_key_id: ')
secret_key = getpass.getpass('aws_secret_access_key: ')
region_name = getpass.getpass('region name: ')

<hr style='height:2px;border:none'>
<p style='font-size:20px;font-family:Arial'><b>3. Use the <code>TextAnalyticsAI</code> API to Perform Various Text Analytics Tasks</b></p>
<p style='font-size:16px;font-family:Arial'>You can execute the help function at the bottom of this notebook to read more about this API.</p>

In [ ]:
# Provide model details
model_name="us.anthropic.claude-3-7-sonnet-20250219-v1:0"

# Select in-database or external model
llm = TeradataAI(api_type = 'AWS',
         model_name = model_name,
         region = region_name,
         # authorization = 'Repositories.BedrockAuth'
         access_key = access_key,
         secret_key = secret_key)

obj = TextAnalyticsAI(llm=llm)

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>4. Complaints Sentiment Analysis</b>
<p style="font-size:16px;font-family:Arial">We'll analyze the sentiments of a sample of customer complaints data.</p>

In [ ]:
tdf = DataFrame(in_schema('DEMO_ComplaintAnalysis', 'Consumer_Complaints'))
tdf

In [ ]:
# let's take sample data
tdf_sample = tdf.sample(100)

In [ ]:
tdf_sentiment = obj.analyze_sentiment(column = 'consumer_complaint_narrative', 
                                      data = tdf_sample)[['date_received','complaint_id','Sentiment','consumer_complaint_narrative', 'product']]

In [ ]:
tdf_sentiment

<p style = 'font-size:16px;font-family:Arial'>Now the results can be saved back to Vantage.</p>

In [ ]:
copy_to_sql(df = tdf_sentiment, table_name = 'complaints_sentiment', if_exists = 'replace')

In [ ]:
sentiment_df = DataFrame('complaints_sentiment')
sentiment_df = sentiment_df.assign(date_received = sentiment_df.date_received.cast(type_=DATE))
sentiment_df = sentiment_df.assign(Sentiment = sentiment_df.Sentiment.str.strip())
print('Before: ', sentiment_df.shape)
sentiment_df = sentiment_df.loc[sentiment_df.Sentiment.isin(['positive', 'negative', 'neutral'])]
print('After: ', sentiment_df.shape)

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial'><b>4.1 Consumer Sentiments Prediction vs Occurrences</b></p>

<p style = 'font-size:16px;font-family:Arial'>A graph illustrating the relationship between consumer sentiments (positive, negative, neutral) prediction and the number of occurrences. This visual representation helps identify trends, patterns, and areas for improvement, enabling data-driven decision making.</p>

In [ ]:
from IPython.display import display, Markdown
def display_helper(msg):
    return display(Markdown(
        f"""<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial'><b>Note: </b>
<i>{msg}</i></p>"""))

In [ ]:
from collections import Counter
import plotly.io as pio
pio.renderers.default = "notebook_connected"

data = Counter(sentiment_df[['Sentiment']].get_values().flatten())

# Convert Counter data to DataFrame
df = pd.DataFrame.from_dict(data, orient='index', columns=['Count']).reset_index()

# Rename columns
df.columns = ['Sentiment', 'Count']

# Create bar graph using Plotly Express
fig = px.bar(df, x='Sentiment', y='Count', color='Sentiment',
             labels={'Count': 'Number of Occurrences', 'Sentiment': 'Sentiment'})

# Show the plot
fig.show()

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial'><b>4.2 Word Cloud for Negative Consumer Sentiment Prediction</b></p>

<p style='font-size:16px;font-family:Arial'> Unlock the power of customer feedback with our intuitive word cloud visualization, which provides a comprehensive snapshot of <b>negative consumer complaints sentiment</b>. This innovative tool highlights the most frequently occurring words and pain points in customer feedback, empowering businesses to: </p> <ol style='font-size:16px;font-family:Arial'> <li>Identify trends and sentiment patterns</li> <li>Pinpoint areas for improvement</li> <li>Make data-driven decisions to enhance customer satisfaction and loyalty</li> </ol> <p style='font-size:16px;font-family:Arial'> By leveraging this word cloud, businesses can proactively address customer concerns, refine their products and services, and ultimately drive growth through a deeper understanding of their customers' needs and preferences. </p>

In [ ]:
neg = sentiment_df[sentiment_df['Sentiment'] == 'negative'].to_pandas()
neg_text = ' '.join(neg['consumer_complaint_narrative'])

# Replace 'X' with blank space
modified_string = neg_text.replace('X', '')

if len(modified_string) > 0:
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(modified_string)

    # Display the word cloud
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.tight_layout()
    plt.axis("off")
    plt.show()
else:
    display_helper("We included positive, negative, and neutral categories to cover all bases. But in this sample, it's possible that none of the complaints are actually negative.")

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial'><b>4.3 Word Cloud for Neutral Consumer Sentiment Prediction</b></p>

<p style='font-size:16px;font-family:Arial'>Tap into the insights of customer feedback with our intuitive word cloud visualization, which offers a detailed overview of <b>neutral consumer complaints sentiment</b></p>

In [ ]:
neu = sentiment_df[sentiment_df['Sentiment'] == 'neutral'].to_pandas()
neu_text = ' '.join(neu['consumer_complaint_narrative'])

# Replace 'X' with blank space
modified_string = neu_text.replace('X', '')

if len(modified_string) > 0:
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(modified_string)

    # Display the word cloud
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.tight_layout()
    plt.axis("off")
    plt.show()
else:
    display_helper("To cover all possible scenarios, we included positive, negative, and neutral categories in our analysis. However, given that this dataset consists of complaints, it's expected that the model would rarely, if ever, encounter positive or neutral responses.")

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial'><b>4.4 Word Cloud for Positive Consumer Sentiment Prediction</b></p>

<p style="font-size:16px;font-family:Arial">Explore customer feedback insights with our intuitive word cloud visualization, providing a detailed overview of consumer sentiment.</p>

In [ ]:
pos = sentiment_df[sentiment_df['Sentiment'] == 'positive'].to_pandas()
pos_text = ' '.join(pos['consumer_complaint_narrative'])

# Replace 'X' with blank space
modified_string = pos_text.replace('X', '')

if len(modified_string) > 0:
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(modified_string)

    # Display the word cloud
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.tight_layout()
    plt.axis("off")
    plt.show()
else:
    display_helper("To cover all possible scenarios, we included positive, negative, and neutral categories in our analysis. However, given that this dataset consists of complaints, it's expected that the model would rarely, if ever, encounter positive or neutral responses.")

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial'><b>4.5 Negative Sentiment per Product Over Years</b></p>

<p style="font-size:16px;font-family:Arial">This graph tracks the negative sentiment  associated with different products over time, offering valuable insights into evolving customer perceptions and pain points.</p>

<p style="font-size:16px;font-family:Arial">We will use <b>Vantage in-db</b> function <b>OrdinalEncodingFit</b> which will identifies distinct categorical values from the input data or a user-defined list and generates the distinct categorical values along with the ordinal value for each category.<p?

In [ ]:
ordinal_fit = OrdinalEncodingFit(
    data = sentiment_df,
    target_column = ['Sentiment'],
    approach = 'LIST',
    categories = ['negative', 'neutral', 'positive']
)

ordinal_fit.result

In [ ]:
out = ColumnTransformer(
    input_data = sentiment_df[['date_received', 'product', 'Sentiment']],
    ordinalencoding_fit_data = ordinal_fit.result
)

In [ ]:
result = out.result
result = result.assign(Sentiment = result.Sentiment - 1)
result = result.assign(year = func.td_year_of_calendar(result.date_received.expression))
result

In [ ]:
viz_neg = result[result['Sentiment'] == -1]

if viz_neg.shape[0] > 0:

    viz_senti = viz_neg.select(['product','Sentiment', 'year']).groupby(['product', 'year']).agg(['sum']).to_pandas()

    # Sorting the DataFrame by year for each product
    pd_df_sorted = viz_senti.sort_values(by=['product', 'year'])

    # Plotting using Plotly
    fig = px.line(pd_df_sorted, x='year', y='sum_Sentiment', color='product', markers=True, title='Negative Sentiment per Product Over Years')
    fig.update_layout(xaxis_title='Year', yaxis_title='Count', legend_title='Product', width=1000, height=600)

    fig.show()
else:
    display_helper("We included positive, negative, and neutral categories to cover all bases. But in this sample, it's possible that none of the complaints are actually negative.")

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial'><b>4.6 Neutral Sentiment per Product Over Years</b></p>

<p style="font-size:16px;font-family:Arial">This graph tracks the neutral sentiment  associated with different products over time, offering valuable insights into evolving customer perceptions and pain points.</p>

In [ ]:
viz_neu = result[result['Sentiment'] == 0]

if viz_neu.shape[0] > 0:
    viz_senti = viz_neu.select(['product','Sentiment', 'year']).groupby(['product', 'year']).agg(['sum']).to_pandas()

    # Sorting the DataFrame by year for each product
    pd_df_sorted = viz_senti.sort_values(by=['product', 'year'])

    # Plotting using Plotly
    fig = px.line(pd_df_sorted, x='year', y='sum_Sentiment', color='product', markers=True, title='Neutral Sentiment per Product Over Years')
    fig.update_layout(xaxis_title='Year', yaxis_title='Count', legend_title='Product', width=1000, height=600)

    fig.show()
else:
    display_helper("To cover all possible scenarios, we included positive, negative, and neutral categories in our analysis. However, given that this dataset consists of complaints, it's expected that the model would rarely, if ever, encounter positive or neutral responses.")

<hr style='height:1px;border:none;'>
<p style = 'font-size:18px;font-family:Arial'><b>4.7 Positive Sentiment per Product Over Years</b></p>

<p style="font-size:16px;font-family:Arial">This graph tracks the positive sentiment  associated with different products over time, offering valuable insights into evolving customer perceptions and pain points.</p>

In [ ]:
viz_pos = result[result['Sentiment'] == 1]

if viz_pos.shape[0] > 0:
    viz_senti = viz_pos.select(['product','Sentiment', 'year']).groupby(['product', 'year']).agg(['sum']).to_pandas()

    # Sorting the DataFrame by year for each product
    pd_df_sorted = viz_senti.sort_values(by=['product', 'year'])

    # Plotting using Plotly
    fig = px.line(pd_df_sorted, x='year', y='sum_Sentiment', color='product', markers=True, title='Positive Sentiment per Product Over Years')
    fig.update_layout(xaxis_title='Year', yaxis_title='Count', legend_title='Product', width=1000, height=600)

    fig.show()
else:
    display_helper("To cover all possible scenarios, we included positive, negative, and neutral categories in our analysis. However, given that this dataset consists of complaints, it's expected that the model would rarely, if ever, encounter positive or neutral responses.")

<hr style="height:2px;border:none;">
<b style = 'font-size:20px;font-family:Arial'>5. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
tables = ['complaints_sentiment']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name=table)
    except:
        pass

<p style = 'font-size:18px;font-family:Arial'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial'>The following code will clean up tables and databases created above.</p>

In [ ]:
remove_context()

<hr style="height:1px;border:none;">
<b style = 'font-size:18px;font-family:Arial'>Dataset:</b>
<br>
<br>
<p style='font-size: 16px; font-family: Arial;'>The dataset is sourced from <a href='https://www.consumerfinance.gov/data-research/consumer-complaints/'>Consumer Financial Protection Bureau</a></p>

<footer style="padding-bottom:35px; border-bottom:3px solid #91A0Ab">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2024, 2025, 2026. All Rights Reserved
        </div>
    </div>
</footer>